In [92]:
import pandas as pd
import numpy as np
from typing import List
from semantic_search.data import build_corpus
from semantic_search.local import LocalKnowledgeBase

def get_base(path: str, model_id: str) -> LocalKnowledgeBase:
    df = pd.read_csv(path)

    codes, texts, titles, descs = [], [], [], []
    for idx, row in df.iterrows():
        desc = row["descriptor"]
        desc_list = split_descriptor(desc) if type(desc) == str else []

        descriptor_template = "#{title}\n{content}.\n\nPercorso: {hierarchy}"

        descriptors = [descriptor_template.format(title=row["title"], content=d, hierarchy=row["hierarchy"]) for d in desc_list]
        texts.extend(descriptors)

        for d in desc_list:
            codes.append(row["code"])
            titles.append(row["title"])
            descs.append(row["descriptor"])
    
    corpus = build_corpus(
        texts=texts,
        ids=list(range(len(texts))),
        metadata=[{"code": c, "title": t, "description": d} for c, t, d in zip(codes, titles, descs)]
    )

    return LocalKnowledgeBase(
        corpus=corpus,
        model_id=model_id,
        batch_size=64
    )


def split_descriptor(text: str) -> List[str]:
    elements = text.split("\n\n") if type(text) == str else []
    
    items = []
    for el in elements:
        if ":\n*" not in el:
            items.append(el.rstrip("\n"))

        else:
            lines = el.strip().splitlines()
            header = ""
            _items = []
            for line in lines:

                if line.startswith("*"):
                    item = line[1:].strip()

                    if header:
                        _items.append(f"{header} {item.lower()}")

                elif line:
                    header = line.rstrip(":")
                
            _items = [i.rstrip("\n") for i in _items]
            items.extend(_items)
    
    return items

def parse_retrieved(results):
    codes = [r.metadata["code"] for r in results[0]]
    titles = [r.metadata["title"] for r in results[0]]
    descriptions = [r.metadata["description"] for r in results[0]]
    texts = [r.text for r in results[0]]
    scores = [r.score for r in results[0]]

    df = pd.DataFrame({
        "code": codes,
        "title": titles,
        "matched_text": texts,
        "description": descriptions,
        "score": scores
    })

    grouped_df = df.groupby("code").aggregate({
        "title": lambda x: np.unique(x)[0],
        "description": lambda x: np.unique(x)[0],
        "score": "max",
    }).reset_index()

    return grouped_df.sort_values("score", ascending=False)


In [71]:
base = get_base("data/ateco_2025_leaves.csv", "BAAI/bge-m3")

Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78/78 [01:20<00:00,  1.03s/it]


In [98]:
query = "Commercio al dettaglio di bevande alcoliche"

results = base.search(query, top_k=10)

df = parse_retrieved(results)
df

Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 72.68it/s]


,code,title,description,score
4,47.25.00,Commercio al dettaglio di bevande,Commercio al dettaglio di bevande alcoliche e ...,0.631006
0,46.34.10,Commercio all'ingrosso di bevande alcoliche,Commercio all'ingrosso di bevande alcoliche\n\...,0.569472
1,46.34.20,Commercio all'ingrosso di bevande non alcoliche,Commercio all'ingrosso di bevande analcoliche\...,0.556228
5,47.27.20,Commercio al dettaglio di caffè,Commercio al dettaglio di caffè e di prodotti ...,0.552604
3,47.24.20,Commercio al dettaglio di pasticceria e dolciumi,Commercio al dettaglio di prodotti di pasticce...,0.550126
2,47.22.00,Commercio al dettaglio di carni e di prodotti ...,Commercio al dettaglio di carni e prodotti a b...,0.546655
6,47.27.90,Commercio al dettaglio di altri prodotti alime...,Commercio al dettaglio di altri prodotti alime...,0.543080
